## Modulo Experiments

This notebook depicts an attempt at getting a neural network to learn a modulo function.

The general consensus is that a neural network can fit any function (Cybenko, 1990) http://cognitivemedium.com/magic_paper/assets/Cybenko.pdf

However, given that the neurons in a traditional neural network are typically only adding or subtracting the input weights * input data value, it is difficult to approximate multiplication and division.

A modulo involves calculating both addition/subtraction and multiplication/division, hence it is fundamentally challenging.

However, can a neural network learn a concept, such as how to calculate a modulo function? Such concepts are typically calculated using a procedural set of instructions, and is not exactly the domain of traditional neural networks.

The modulo function is basically the remainder. Some examples are:
- 5 % 2 = 1
- 5 % 3 = 2
- 6 % 2 = 0

One example of a set of instructions for modulo of a % b is as follows:
1) Calculate c = a//b (the floor of a/b)
2) Calculate modulo = a - c*b

Dependencies:
- tensorflow
- numpy

## Inputs/Outputs

Train Inputs:
250000 random integers from 0 to 2^20 (binarized)

Test Inputs:
10000 random integers from 2^20 to 2^21 (binarized)

Outputs:
Modulo of the input with factor 7

## Base Model
The base model in this notebook uses 3 different models:
- MLP with 1 hidden layer of 1000 nodes with ReLU activation, output softmax layer of 7 nodes
- MLP with 2 hidden layers of 1000 nodes with ReLU activation, output softmax layer of 7 nodes
- a ResNet with 3 layers, with skip connections between all layers, output softmax layer of 7 nodes

## Baseline Results
MLP with 1/2 hidden layers of 1000 nodes, ResNet: 
We are able to get a train accuracy of up to 100%, but the test accuracy tends to 0%!
Strong overfitting here.
This also means that the model is unable to generalize.

# What can we do?
We try five approaches:
- Add divisor as input
- Using whole numbers instead of binary as input
- Few-shot learning
- One-shot learning
- Zero-shot learning with X-factor

# Binary Representation (Baseline Results)

In [5]:
import tensorflow as tf, numpy as np

# hyperparameters here
divisor = 7

# convert a number into binary
def int2bits(i,fill=21): 
    return list(map(int,bin(i)[2:].zfill(fill)))

def bits2int(b):
    return sum(i*2**n for n,i in enumerate(reversed(b)))

# Data. 
I = np.random.randint(0,2**20,size=(250000,))
X = np.array(list(map(int2bits,I)))
Y = np.array([int2bits(2**i,divisor) for i in I % divisor])

# Test Data. 
It = np.random.randint(2**20,2**21,size=(10000,))
Xt = np.array(list(map(int2bits,It)))
Yt = np.array([int2bits(2**i,divisor) for i in It % divisor])

# MLP

In [3]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (21))

x = Dense(1000, 'relu')(inputs)
outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,20,validation_data=(Xt,Yt))

Epoch 1/20
25/25 [==============================] - 2s 60ms/step - loss: 1.9480 - accuracy: 0.1433 - val_loss: 1.9484 - val_accuracy: 0.1405
Epoch 2/20
25/25 [==============================] - 1s 57ms/step - loss: 1.9460 - accuracy: 0.1480 - val_loss: 1.9479 - val_accuracy: 0.1406
Epoch 3/20
25/25 [==============================] - 1s 58ms/step - loss: 1.9453 - accuracy: 0.1514 - val_loss: 1.9479 - val_accuracy: 0.1399
Epoch 4/20
25/25 [==============================] - 1s 57ms/step - loss: 1.9444 - accuracy: 0.1546 - val_loss: 1.9485 - val_accuracy: 0.1408
Epoch 5/20
25/25 [==============================] - 1s 58ms/step - loss: 1.9433 - accuracy: 0.1596 - val_loss: 1.9486 - val_accuracy: 0.1510
Epoch 6/20
25/25 [==============================] - 1s 57ms/step - loss: 1.9417 - accuracy: 0.1656 - val_loss: 1.9506 - val_accuracy: 0.1411
Epoch 7/20
25/25 [==============================] - 1s 57ms/step - loss: 1.9393 - accuracy: 0.1751 - val_loss: 1.9510 - val_accuracy: 0.1412
Epoch 8/20
25

In [6]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (21))
# x = Dense(1000, 'relu')(inputs)

# # Do a ResNet style skip connection
# layer1 = Concatenate()([x, inputs])
# x = Dense(1000, 'relu')(layer1)

# # Do a double skip connection
# layer2 = Concatenate()([x, layer1])
# x = Dense(1000, 'relu')(layer2)

# # Do a triple skip connection
# layer3 = Concatenate()([x, layer2])
# x = Dense(1000, 'relu')(layer3)

x = Dense(1000, 'relu')(inputs)
x = Dense(1000, 'relu')(x)
outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,20,validation_data=(Xt,Yt))

Epoch 1/20
25/25 [==============================] - 4s 160ms/step - loss: 1.9513 - accuracy: 0.1440 - val_loss: 1.9468 - val_accuracy: 0.1424
Epoch 2/20
25/25 [==============================] - 4s 156ms/step - loss: 1.9455 - accuracy: 0.1475 - val_loss: 1.9467 - val_accuracy: 0.1396
Epoch 3/20
25/25 [==============================] - 4s 166ms/step - loss: 1.9447 - accuracy: 0.1540 - val_loss: 1.9470 - val_accuracy: 0.1401
Epoch 4/20
25/25 [==============================] - 4s 175ms/step - loss: 1.9438 - accuracy: 0.1576 - val_loss: 1.9477 - val_accuracy: 0.1453
Epoch 5/20
25/25 [==============================] - 4s 177ms/step - loss: 1.9424 - accuracy: 0.1639 - val_loss: 1.9477 - val_accuracy: 0.1405
Epoch 6/20
25/25 [==============================] - 4s 175ms/step - loss: 1.9397 - accuracy: 0.1702 - val_loss: 1.9481 - val_accuracy: 0.1408
Epoch 7/20
25/25 [==============================] - 4s 175ms/step - loss: 1.9347 - accuracy: 0.1824 - val_loss: 1.9490 - val_accuracy: 0.1402
Epoch 

In [7]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (21))
# x = Dense(1000, 'relu')(inputs)

# # Do a ResNet style skip connection
# layer1 = Concatenate()([x, inputs])
# x = Dense(1000, 'relu')(layer1)

# # Do a double skip connection
# layer2 = Concatenate()([x, layer1])
# x = Dense(1000, 'relu')(layer2)

# # Do a triple skip connection
# layer3 = Concatenate()([x, layer2])
# x = Dense(1000, 'relu')(layer3)

x = Dense(1000, 'relu')(inputs)
x = Dense(1000, 'relu')(x)
x = Dense(1000, 'relu')(x)
outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,20,validation_data=(Xt,Yt))

Epoch 1/20
25/25 [==============================] - 9s 370ms/step - loss: 1.9498 - accuracy: 0.1429 - val_loss: 1.9463 - val_accuracy: 0.1426
Epoch 2/20
25/25 [==============================] - 9s 351ms/step - loss: 1.9457 - accuracy: 0.1471 - val_loss: 1.9462 - val_accuracy: 0.1429
Epoch 3/20
25/25 [==============================] - 9s 344ms/step - loss: 1.9452 - accuracy: 0.1503 - val_loss: 1.9467 - val_accuracy: 0.1392
Epoch 4/20
25/25 [==============================] - 8s 339ms/step - loss: 1.9447 - accuracy: 0.1526 - val_loss: 1.9466 - val_accuracy: 0.1438
Epoch 5/20
25/25 [==============================] - 9s 354ms/step - loss: 1.9433 - accuracy: 0.1595 - val_loss: 1.9481 - val_accuracy: 0.1466
Epoch 6/20
25/25 [==============================] - 9s 374ms/step - loss: 1.9408 - accuracy: 0.1665 - val_loss: 1.9520 - val_accuracy: 0.1375
Epoch 7/20
25/25 [==============================] - 9s 378ms/step - loss: 1.9311 - accuracy: 0.1832 - val_loss: 1.9710 - val_accuracy: 0.1259
Epoch 

# ResNet

In [8]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (21))
x = Dense(1000, 'relu')(inputs)

# Do a ResNet style skip connection
layer1 = Concatenate()([x, inputs])
x = Dense(1000, 'relu')(layer1)

# Do a double skip connection
layer2 = Concatenate()([x, layer1])
x = Dense(1000, 'relu')(layer2)

# Do a triple skip connection
layer3 = Concatenate()([x, layer2])
x = Dense(1000, 'relu')(layer3)

# x = Dense(1000, 'relu')(inputs)
# x = Dense(1000, 'relu')(x)
# x = Dense(1000, 'relu')(x)
outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,20,validation_data=(Xt,Yt))

Epoch 1/20
25/25 [==============================] - 29s 1s/step - loss: 1.9876 - accuracy: 0.1429 - val_loss: 1.9462 - val_accuracy: 0.1430
Epoch 2/20
25/25 [==============================] - 30s 1s/step - loss: 1.9459 - accuracy: 0.1453 - val_loss: 1.9461 - val_accuracy: 0.1382
Epoch 3/20
25/25 [==============================] - 29s 1s/step - loss: 1.9457 - accuracy: 0.1467 - val_loss: 1.9461 - val_accuracy: 0.1470
Epoch 4/20
25/25 [==============================] - 29s 1s/step - loss: 1.9455 - accuracy: 0.1494 - val_loss: 1.9462 - val_accuracy: 0.1374
Epoch 5/20
25/25 [==============================] - 31s 1s/step - loss: 1.9452 - accuracy: 0.1516 - val_loss: 1.9463 - val_accuracy: 0.1383
Epoch 6/20
25/25 [==============================] - 30s 1s/step - loss: 1.9447 - accuracy: 0.1560 - val_loss: 1.9472 - val_accuracy: 0.1381
Epoch 7/20
25/25 [==============================] - 29s 1s/step - loss: 1.9435 - accuracy: 0.1609 - val_loss: 1.9489 - val_accuracy: 0.1418
Epoch 8/20
25/25 [==

# Conclusion: Binary inputs with MLP, ResNet doesn't work!

# How about binary with divisor as input?

In [208]:
import tensorflow as tf, numpy as np

# hyperparameters here
divisor = 7

# convert a number into binary
def int2bits(i,fill=24): 
    return list(map(int,bin(i)[2:].zfill(fill)))

def bits2int(b):
    return sum(i*2**n for n,i in enumerate(reversed(b)))

# Data. 
I = np.random.randint(0,2**20,size=(250000,))
X = np.array(list(map(int2bits,I*8+divisor)))
Y = np.array([int2bits(2**i,divisor) for i in I % divisor])

# Test Data. 
It = np.random.randint(2**20,2**21,size=(10000,))
Xt = np.array(list(map(int2bits,It*8+divisor)))
Yt = np.array([int2bits(2**i,divisor) for i in It % divisor])

In [210]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (24))

x = Dense(100, 'relu')(inputs)
outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,20,validation_data=(Xt,Yt))

Epoch 1/20
25/25 [==============================] - 0s 10ms/step - loss: 1.9773 - accuracy: 0.1417 - val_loss: 1.9585 - val_accuracy: 0.1474
Epoch 2/20
25/25 [==============================] - 0s 7ms/step - loss: 1.9549 - accuracy: 0.1422 - val_loss: 1.9550 - val_accuracy: 0.1448
Epoch 3/20
25/25 [==============================] - 0s 7ms/step - loss: 1.9504 - accuracy: 0.1429 - val_loss: 1.9534 - val_accuracy: 0.1477
Epoch 4/20
25/25 [==============================] - 0s 7ms/step - loss: 1.9486 - accuracy: 0.1434 - val_loss: 1.9524 - val_accuracy: 0.1468
Epoch 5/20
25/25 [==============================] - 0s 7ms/step - loss: 1.9479 - accuracy: 0.1438 - val_loss: 1.9533 - val_accuracy: 0.1464
Epoch 6/20
25/25 [==============================] - 0s 8ms/step - loss: 1.9473 - accuracy: 0.1460 - val_loss: 1.9532 - val_accuracy: 0.1455
Epoch 7/20
25/25 [==============================] - 0s 7ms/step - loss: 1.9469 - accuracy: 0.1465 - val_loss: 1.9518 - val_accuracy: 0.1453
Epoch 8/20
25/25 [=

In [ ]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (21))
# x = Dense(1000, 'relu')(inputs)

# # Do a ResNet style skip connection
# layer1 = Concatenate()([x, inputs])
# x = Dense(1000, 'relu')(layer1)

# # Do a double skip connection
# layer2 = Concatenate()([x, layer1])
# x = Dense(1000, 'relu')(layer2)

# # Do a triple skip connection
# layer3 = Concatenate()([x, layer2])
# x = Dense(1000, 'relu')(layer3)

x = Dense(1000, 'relu')(inputs)
outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,20,validation_data=(Xt,Yt))

Epoch 1/20
25/25 [==============================] - 2s 60ms/step - loss: 1.9480 - accuracy: 0.1433 - val_loss: 1.9484 - val_accuracy: 0.1405
Epoch 2/20
25/25 [==============================] - 1s 57ms/step - loss: 1.9460 - accuracy: 0.1480 - val_loss: 1.9479 - val_accuracy: 0.1406
Epoch 3/20
25/25 [==============================] - 1s 58ms/step - loss: 1.9453 - accuracy: 0.1514 - val_loss: 1.9479 - val_accuracy: 0.1399
Epoch 4/20
25/25 [==============================] - 1s 57ms/step - loss: 1.9444 - accuracy: 0.1546 - val_loss: 1.9485 - val_accuracy: 0.1408
Epoch 5/20
25/25 [==============================] - ETA: 0s - loss: 1.9433 - accuracy: 0.1596

In [20]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (24))

x = Dense(1000, 'relu')(inputs)
outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,20,validation_data=(Xt,Yt))

Epoch 1/20
25/25 [==============================] - 1s 58ms/step - loss: 1.9484 - accuracy: 0.1432 - val_loss: 1.9472 - val_accuracy: 0.1469
Epoch 2/20
25/25 [==============================] - 1s 54ms/step - loss: 1.9464 - accuracy: 0.1466 - val_loss: 1.9472 - val_accuracy: 0.1448
Epoch 3/20
25/25 [==============================] - 1s 54ms/step - loss: 1.9456 - accuracy: 0.1487 - val_loss: 1.9482 - val_accuracy: 0.1475
Epoch 4/20
25/25 [==============================] - 1s 54ms/step - loss: 1.9449 - accuracy: 0.1530 - val_loss: 1.9473 - val_accuracy: 0.1427
Epoch 5/20
25/25 [==============================] - 1s 54ms/step - loss: 1.9439 - accuracy: 0.1561 - val_loss: 1.9479 - val_accuracy: 0.1493
Epoch 6/20
25/25 [==============================] - 1s 54ms/step - loss: 1.9427 - accuracy: 0.1626 - val_loss: 1.9480 - val_accuracy: 0.1444
Epoch 7/20
25/25 [==============================] - 1s 55ms/step - loss: 1.9414 - accuracy: 0.1672 - val_loss: 1.9485 - val_accuracy: 0.1460
Epoch 8/20
25

In [21]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (24))

x = Dense(1000, 'relu')(inputs)
x = Dense(1000, 'relu')(x)
outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,20,validation_data=(Xt,Yt))

Epoch 1/20
25/25 [==============================] - 4s 158ms/step - loss: 1.9531 - accuracy: 0.1431 - val_loss: 1.9463 - val_accuracy: 0.1421
Epoch 2/20
25/25 [==============================] - 4s 155ms/step - loss: 1.9458 - accuracy: 0.1459 - val_loss: 1.9459 - val_accuracy: 0.1488
Epoch 3/20
25/25 [==============================] - 4s 164ms/step - loss: 1.9451 - accuracy: 0.1515 - val_loss: 1.9457 - val_accuracy: 0.1476
Epoch 4/20
25/25 [==============================] - 4s 174ms/step - loss: 1.9437 - accuracy: 0.1600 - val_loss: 1.9459 - val_accuracy: 0.1576
Epoch 5/20
25/25 [==============================] - 4s 172ms/step - loss: 1.9413 - accuracy: 0.1707 - val_loss: 1.9465 - val_accuracy: 0.1483
Epoch 6/20
25/25 [==============================] - 4s 175ms/step - loss: 1.9363 - accuracy: 0.1830 - val_loss: 1.9486 - val_accuracy: 0.1504
Epoch 7/20
25/25 [==============================] - 4s 175ms/step - loss: 1.9268 - accuracy: 0.2054 - val_loss: 1.9542 - val_accuracy: 0.1468
Epoch 

# ResNet

In [23]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (24))
x = Dense(1000, 'relu')(inputs)

# Do a ResNet style skip connection
layer1 = Concatenate()([x, inputs])
x = Dense(1000, 'relu')(layer1)

# Do a double skip connection
layer2 = Concatenate()([x, layer1])
x = Dense(1000, 'relu')(layer2)

# Do a triple skip connection
layer3 = Concatenate()([x, layer2])
x = Dense(1000, 'relu')(layer3)

outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,20,validation_data=(Xt,Yt))

Epoch 1/20
25/25 [==============================] - 25s 995ms/step - loss: 1.9893 - accuracy: 0.1443 - val_loss: 1.9464 - val_accuracy: 0.1455
Epoch 2/20
25/25 [==============================] - 35s 1s/step - loss: 1.9461 - accuracy: 0.1452 - val_loss: 1.9459 - val_accuracy: 0.1431
Epoch 3/20
25/25 [==============================] - 33s 1s/step - loss: 1.9458 - accuracy: 0.1467 - val_loss: 1.9459 - val_accuracy: 0.1425
Epoch 4/20
25/25 [==============================] - 29s 1s/step - loss: 1.9455 - accuracy: 0.1491 - val_loss: 1.9460 - val_accuracy: 0.1458
Epoch 5/20
25/25 [==============================] - 31s 1s/step - loss: 1.9453 - accuracy: 0.1500 - val_loss: 1.9464 - val_accuracy: 0.1394
Epoch 6/20
25/25 [==============================] - 30s 1s/step - loss: 1.9450 - accuracy: 0.1518 - val_loss: 1.9461 - val_accuracy: 0.1466
Epoch 7/20
25/25 [==============================] - 31s 1s/step - loss: 1.9443 - accuracy: 0.1557 - val_loss: 1.9461 - val_accuracy: 0.1510
Epoch 8/20
25/25 

# Conclusion: Binary with divisor as input doesn't work either

# Whole number with divisor as input

In [36]:
import tensorflow as tf, numpy as np

# hyperparameters here
divisor = 7

# Data. 
I = np.random.randint(0,2**20,size=(250000,))
X = np.array([[i, divisor] for i in I])
Y = np.array([[1 if element%divisor == i else 0 for i in range(7)] for element in I])

# Test Data. 
It = np.random.randint(0,2**20,size=(250000,))
Xt = np.array([[i, divisor] for i in It])
Yt = np.array([[1 if element%divisor == i else 0 for i in range(7)] for element in It])

In [37]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model

### Change the model architecture here
########################################################
inputs = Input(shape = (2))

x = Dense(1000, 'relu')(inputs)
outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,20,validation_data=(Xt,Yt))

Epoch 1/20
25/25 [==============================] - 2s 86ms/step - loss: 12793.1797 - accuracy: 0.1433 - val_loss: 3619.6604 - val_accuracy: 0.1442
Epoch 2/20
25/25 [==============================] - 2s 83ms/step - loss: 4156.2104 - accuracy: 0.1405 - val_loss: 3378.4050 - val_accuracy: 0.1430
Epoch 3/20
25/25 [==============================] - 2s 84ms/step - loss: 4391.4160 - accuracy: 0.1430 - val_loss: 3852.6606 - val_accuracy: 0.1430
Epoch 4/20
25/25 [==============================] - 2s 83ms/step - loss: 4234.9595 - accuracy: 0.1434 - val_loss: 4352.6245 - val_accuracy: 0.1432
Epoch 5/20
25/25 [==============================] - 2s 82ms/step - loss: 4386.7671 - accuracy: 0.1434 - val_loss: 5566.9521 - val_accuracy: 0.1429
Epoch 6/20
25/25 [==============================] - 2s 83ms/step - loss: 4512.9819 - accuracy: 0.1431 - val_loss: 4843.7583 - val_accuracy: 0.1433
Epoch 7/20
25/25 [==============================] - 2s 84ms/step - loss: 4509.7236 - accuracy: 0.1414 - val_loss: 500

In [38]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model

### Change the model architecture here
########################################################
inputs = Input(shape = (2))

x = Dense(1000, 'relu')(inputs)
x = Dense(1000, 'relu')(inputs)
outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,20,validation_data=(Xt,Yt))

Epoch 1/20
25/25 [==============================] - 2s 87ms/step - loss: 16569.1738 - accuracy: 0.1419 - val_loss: 9125.2305 - val_accuracy: 0.1425
Epoch 2/20
25/25 [==============================] - 2s 84ms/step - loss: 5992.2178 - accuracy: 0.1429 - val_loss: 8336.6709 - val_accuracy: 0.1442
Epoch 3/20
25/25 [==============================] - 2s 84ms/step - loss: 6385.6172 - accuracy: 0.1423 - val_loss: 6814.0874 - val_accuracy: 0.1442
Epoch 4/20
25/25 [==============================] - 2s 84ms/step - loss: 4724.9722 - accuracy: 0.1426 - val_loss: 3587.0002 - val_accuracy: 0.1429
Epoch 5/20
25/25 [==============================] - 2s 85ms/step - loss: 4345.5430 - accuracy: 0.1416 - val_loss: 4324.8667 - val_accuracy: 0.1432
Epoch 6/20
25/25 [==============================] - 2s 86ms/step - loss: 4946.7236 - accuracy: 0.1427 - val_loss: 3511.1323 - val_accuracy: 0.1442
Epoch 7/20
25/25 [==============================] - 2s 85ms/step - loss: 4558.7979 - accuracy: 0.1428 - val_loss: 541

In [40]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (2))
x = Dense(1000, 'relu')(inputs)

# Do a ResNet style skip connection
layer1 = Concatenate()([x, inputs])
x = Dense(1000, 'relu')(layer1)

# Do a double skip connection
layer2 = Concatenate()([x, layer1])
x = Dense(1000, 'relu')(layer2)

# Do a triple skip connection
layer3 = Concatenate()([x, layer2])
x = Dense(1000, 'relu')(layer3)

outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,20,validation_data=(Xt,Yt))

Epoch 1/20
25/25 [==============================] - 31s 1s/step - loss: 104442.2031 - accuracy: 0.1434 - val_loss: 16970.2012 - val_accuracy: 0.1430
Epoch 2/20
25/25 [==============================] - 32s 1s/step - loss: 16606.9453 - accuracy: 0.1435 - val_loss: 15173.5703 - val_accuracy: 0.1414
Epoch 3/20
25/25 [==============================] - 40s 2s/step - loss: 7759.9438 - accuracy: 0.1426 - val_loss: 8116.0366 - val_accuracy: 0.1442
Epoch 4/20
25/25 [==============================] - 38s 2s/step - loss: 5930.2632 - accuracy: 0.1432 - val_loss: 4046.5327 - val_accuracy: 0.1433
Epoch 5/20
25/25 [==============================] - 37s 1s/step - loss: 5922.5200 - accuracy: 0.1432 - val_loss: 8316.2031 - val_accuracy: 0.1428
Epoch 6/20
25/25 [==============================] - 37s 1s/step - loss: 4896.7603 - accuracy: 0.1437 - val_loss: 4244.5918 - val_accuracy: 0.1414
Epoch 7/20
25/25 [==============================] - 38s 2s/step - loss: 4443.0264 - accuracy: 0.1435 - val_loss: 6907.1

# Conclusion: Whole number with divisor as input doesn't work even when testing distribution is same as training distribution

# Could it be we need few-shot learning?

From the earlier experiments, the neural network could not figure out how to handle the 21st bit, but has no issues handling bits 0 to 20. Maybe we need to give it some information about the 21st bit for it to generalize.

Hence, here, other than the 250000 random training inputs from 0 to 2^20, we also insert 100 training inputs from 2^20 to 2^21

In [55]:
import tensorflow as tf, numpy as np

# hyperparameters here
divisor = 7

# convert a number into binary
def int2bits(i,fill=21): 
    return list(map(int,bin(i)[2:].zfill(fill)))

def bits2int(b):
    return sum(i*2**n for n,i in enumerate(reversed(b)))

# Data. 
I = np.append(np.random.randint(0,2**20,size=(250000,)), np.random.randint(2**20, 2**21, size = (100,)))
X = np.array(list(map(int2bits,I)))
Y = np.array([int2bits(2**i,divisor) for i in I % divisor])

# Test Data. 
It = np.random.randint(2**20,2**21,size=(10000,))
Xt = np.array(list(map(int2bits,It)))
Yt = np.array([int2bits(2**i,divisor) for i in It % divisor])

In [57]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (21))

x = Dense(1000, 'relu')(inputs)
outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,100,validation_data=(Xt,Yt))

Epoch 1/100
26/26 [==============================] - 1s 57ms/step - loss: 1.9485 - accuracy: 0.1408 - val_loss: 1.9493 - val_accuracy: 0.1464
Epoch 2/100
26/26 [==============================] - 1s 54ms/step - loss: 1.9470 - accuracy: 0.1462 - val_loss: 1.9481 - val_accuracy: 0.1426
Epoch 3/100
26/26 [==============================] - 1s 55ms/step - loss: 1.9464 - accuracy: 0.1492 - val_loss: 1.9497 - val_accuracy: 0.1450
Epoch 4/100
26/26 [==============================] - 1s 54ms/step - loss: 1.9454 - accuracy: 0.1512 - val_loss: 1.9500 - val_accuracy: 0.1451
Epoch 5/100
26/26 [==============================] - 1s 53ms/step - loss: 1.9452 - accuracy: 0.1526 - val_loss: 1.9505 - val_accuracy: 0.1457
Epoch 6/100
26/26 [==============================] - 1s 54ms/step - loss: 1.9437 - accuracy: 0.1560 - val_loss: 1.9519 - val_accuracy: 0.1467
Epoch 7/100
26/26 [==============================] - 1s 54ms/step - loss: 1.9425 - accuracy: 0.1611 - val_loss: 1.9505 - val_accuracy: 0.1480
Epoch 

In [58]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (21))

x = Dense(1000, 'relu')(inputs)
x = Dense(1000, 'relu')(x)
outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,100,validation_data=(Xt,Yt))

Epoch 1/100
26/26 [==============================] - 4s 154ms/step - loss: 1.9510 - accuracy: 0.1436 - val_loss: 1.9468 - val_accuracy: 0.1441
Epoch 2/100
26/26 [==============================] - 4s 151ms/step - loss: 1.9459 - accuracy: 0.1465 - val_loss: 1.9461 - val_accuracy: 0.1477
Epoch 3/100
26/26 [==============================] - 4s 152ms/step - loss: 1.9453 - accuracy: 0.1503 - val_loss: 1.9463 - val_accuracy: 0.1467
Epoch 4/100
26/26 [==============================] - 4s 162ms/step - loss: 1.9449 - accuracy: 0.1530 - val_loss: 1.9464 - val_accuracy: 0.1426
Epoch 5/100
26/26 [==============================] - 4s 167ms/step - loss: 1.9444 - accuracy: 0.1537 - val_loss: 1.9473 - val_accuracy: 0.1380
Epoch 6/100
26/26 [==============================] - 4s 165ms/step - loss: 1.9436 - accuracy: 0.1582 - val_loss: 1.9475 - val_accuracy: 0.1422
Epoch 7/100
26/26 [==============================] - 4s 165ms/step - loss: 1.9415 - accuracy: 0.1660 - val_loss: 1.9509 - val_accuracy: 0.1401

In [60]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (21))
x = Dense(1000, 'relu')(inputs)

# Do a ResNet style skip connection
layer1 = Concatenate()([x, inputs])
x = Dense(1000, 'relu')(layer1)

# Do a double skip connection
layer2 = Concatenate()([x, layer1])
x = Dense(1000, 'relu')(layer2)

# Do a triple skip connection
layer3 = Concatenate()([x, layer2])
x = Dense(1000, 'relu')(layer3)

outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,100,validation_data=(Xt,Yt))

Epoch 1/100
26/26 [==============================] - 30s 1s/step - loss: 1.9900 - accuracy: 0.1430 - val_loss: 1.9460 - val_accuracy: 0.1436
Epoch 2/100
26/26 [==============================] - 30s 1s/step - loss: 1.9460 - accuracy: 0.1449 - val_loss: 1.9463 - val_accuracy: 0.1379
Epoch 3/100
26/26 [==============================] - 34s 1s/step - loss: 1.9460 - accuracy: 0.1439 - val_loss: 1.9461 - val_accuracy: 0.1412
Epoch 4/100
26/26 [==============================] - 36s 1s/step - loss: 1.9459 - accuracy: 0.1457 - val_loss: 1.9460 - val_accuracy: 0.1508
Epoch 5/100
26/26 [==============================] - 30s 1s/step - loss: 1.9458 - accuracy: 0.1464 - val_loss: 1.9460 - val_accuracy: 0.1405
Epoch 6/100
26/26 [==============================] - 30s 1s/step - loss: 1.9457 - accuracy: 0.1464 - val_loss: 1.9462 - val_accuracy: 0.1371
Epoch 7/100
26/26 [==============================] - 30s 1s/step - loss: 1.9457 - accuracy: 0.1471 - val_loss: 1.9462 - val_accuracy: 0.1395
Epoch 8/100
2

# Conclusion: Few-shot learning works. Can achieve about 55% test accuracy for 1 layer of MLP, 88% test accuracy for 2 layers of MLP, and even 98% test accuracy for ResNet. We did it!

# How about one-shot learning?

Here we only give one out-of-sample training sample to let the neural network learn how to handle the 21st bit.

In [61]:
import tensorflow as tf, numpy as np

# hyperparameters here
divisor = 7

# convert a number into binary
def int2bits(i,fill=21): 
    return list(map(int,bin(i)[2:].zfill(fill)))

def bits2int(b):
    return sum(i*2**n for n,i in enumerate(reversed(b)))

# Data. 
I = np.append(np.random.randint(0,2**20,size=(250000,)), np.random.randint(2**20, 2**21, size = (1,)))
X = np.array(list(map(int2bits,I)))
Y = np.array([int2bits(2**i,divisor) for i in I % divisor])

# Test Data. 
It = np.random.randint(2**20,2**21,size=(10000,))
Xt = np.array(list(map(int2bits,It)))
Yt = np.array([int2bits(2**i,divisor) for i in It % divisor])

In [62]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (21))

x = Dense(1000, 'relu')(inputs)
outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,100,validation_data=(Xt,Yt))

Epoch 1/100
26/26 [==============================] - 2s 61ms/step - loss: 1.9479 - accuracy: 0.1430 - val_loss: 1.9479 - val_accuracy: 0.1502
Epoch 2/100
26/26 [==============================] - 1s 55ms/step - loss: 1.9471 - accuracy: 0.1467 - val_loss: 1.9485 - val_accuracy: 0.1492
Epoch 3/100
26/26 [==============================] - 1s 56ms/step - loss: 1.9485 - accuracy: 0.1470 - val_loss: 1.9506 - val_accuracy: 0.1480
Epoch 4/100
26/26 [==============================] - 1s 55ms/step - loss: 1.9496 - accuracy: 0.1461 - val_loss: 1.9503 - val_accuracy: 0.1515
Epoch 5/100
26/26 [==============================] - 1s 55ms/step - loss: 1.9507 - accuracy: 0.1477 - val_loss: 1.9517 - val_accuracy: 0.1448
Epoch 6/100
26/26 [==============================] - 1s 54ms/step - loss: 1.9534 - accuracy: 0.1456 - val_loss: 1.9544 - val_accuracy: 0.1481
Epoch 7/100
26/26 [==============================] - 1s 55ms/step - loss: 1.9539 - accuracy: 0.1460 - val_loss: 1.9559 - val_accuracy: 0.1503
Epoch 

In [63]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (21))

x = Dense(1000, 'relu')(inputs)
x = Dense(1000, 'relu')(x)
outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,100,validation_data=(Xt,Yt))

Epoch 1/100
26/26 [==============================] - 5s 178ms/step - loss: 1.9516 - accuracy: 0.1437 - val_loss: 1.9476 - val_accuracy: 0.1416
Epoch 2/100
26/26 [==============================] - 5s 191ms/step - loss: 1.9466 - accuracy: 0.1463 - val_loss: 1.9469 - val_accuracy: 0.1407
Epoch 3/100
26/26 [==============================] - 5s 202ms/step - loss: 1.9465 - accuracy: 0.1469 - val_loss: 1.9468 - val_accuracy: 0.1492
Epoch 4/100
26/26 [==============================] - 5s 211ms/step - loss: 1.9459 - accuracy: 0.1490 - val_loss: 1.9472 - val_accuracy: 0.1453
Epoch 5/100
26/26 [==============================] - 5s 207ms/step - loss: 1.9456 - accuracy: 0.1505 - val_loss: 1.9473 - val_accuracy: 0.1370
Epoch 6/100
26/26 [==============================] - 5s 208ms/step - loss: 1.9449 - accuracy: 0.1511 - val_loss: 1.9492 - val_accuracy: 0.1380
Epoch 7/100
26/26 [==============================] - 6s 224ms/step - loss: 1.9450 - accuracy: 0.1542 - val_loss: 1.9474 - val_accuracy: 0.1481

In [64]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (21))
x = Dense(1000, 'relu')(inputs)

# Do a ResNet style skip connection
layer1 = Concatenate()([x, inputs])
x = Dense(1000, 'relu')(layer1)

# Do a double skip connection
layer2 = Concatenate()([x, layer1])
x = Dense(1000, 'relu')(layer2)

# Do a triple skip connection
layer3 = Concatenate()([x, layer2])
x = Dense(1000, 'relu')(layer3)

outputs = Dense(divisor, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10_000,100,validation_data=(Xt,Yt))

Epoch 1/100
26/26 [==============================] - 33s 1s/step - loss: 1.9788 - accuracy: 0.1436 - val_loss: 1.9464 - val_accuracy: 0.1385
Epoch 2/100
26/26 [==============================] - 34s 1s/step - loss: 1.9471 - accuracy: 0.1420 - val_loss: 1.9461 - val_accuracy: 0.1504
Epoch 3/100
26/26 [==============================] - 33s 1s/step - loss: 1.9468 - accuracy: 0.1455 - val_loss: 1.9471 - val_accuracy: 0.1414
Epoch 4/100
26/26 [==============================] - 34s 1s/step - loss: 1.9470 - accuracy: 0.1440 - val_loss: 1.9464 - val_accuracy: 0.1450
Epoch 5/100
26/26 [==============================] - 33s 1s/step - loss: 1.9469 - accuracy: 0.1471 - val_loss: 1.9466 - val_accuracy: 0.1497
Epoch 6/100
26/26 [==============================] - 33s 1s/step - loss: 1.9467 - accuracy: 0.1461 - val_loss: 1.9482 - val_accuracy: 0.1366
Epoch 7/100
26/26 [==============================] - 33s 1s/step - loss: 1.9473 - accuracy: 0.1480 - val_loss: 1.9484 - val_accuracy: 0.1497
Epoch 8/100
2

# Conclusion: One-shot learning works, but converges slower than few-shot learning. It achieves 15/31/89% accuracy for 1-layer/2-layer/ResNet accordingly

# Could zero-shot learning work if we give it the structure to consider?

The first few experiments show that we would require at least one training set to be in the out-of-sample in order for the neural network to generalize. But we have not solved the original task yet. Can we use zero-shot learning in order to let the network really learn modulo?

I could not do it for generic modulo, but for modulo 7, I realize a small trick.
We can view the input as chunks of 3. The first 3 bits will have the same modulo 7 as the second 3 bits, and so on. 

This can be seen that 1 = 8 (mod 7), and in general for any positive integer a, a = 8a = 7a + a = a (mod 7). Hence, a bit shift of 3 to the left (same as multiplying by 8), will have no impact on modulo 7, and can be computed the same way also.

Hence, if we could somehow utilize the network generalization for the first few chunks of 3 bits, it could perhaps use those same mechanisms to evaluate the 21st bit.

That is how I thought of using Conv1D to do chunks of 3 for the binarized input, in order to have the same weights (same method of evaluating modulo), and it can generalize to any chunk of 3.

Further, we then use the neural network speciality to generalize these outputs form the chunks of 3 bits into modulo.

And it worked!

In [190]:
import tensorflow as tf, numpy as np

# hyperparameters here
divisor = 7

# convert a number into binary
def int2bits(i,fill=21): 
    return list(map(int,bin(i)[2:].zfill(fill)))

def bits2int(b):
    return sum(i*2**n for n,i in enumerate(reversed(b)))

# Data. 
I = np.random.randint(0,2**20,size=(250000,))
X = np.array(list(map(int2bits, I))) 
Y = np.array([int2bits(2**i,divisor) for i in I % divisor])

# Test Data. 
It = np.random.randint(2**20,2**21,size=(10000,))
Xt = np.array(list(map(int2bits,It)))
Yt = np.array([int2bits(2**i,divisor) for i in It % divisor])

In [205]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate, Reshape, Conv1D, Flatten
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (21))

x = Reshape((-1, 1))(inputs)
print(x.shape)

x = Conv1D(filters = 1, kernel_size = 3, strides = 3)(x)
print(x.shape)
x = Flatten()(x)
x = Dense(1000, 'relu')(x)
outputs = Dense(7, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10000,100,validation_data=(Xt,Yt))

(None, 21, 1)
(None, 7, 1)
Epoch 1/100
25/25 [==============================] - 2s 71ms/step - loss: 1.9465 - accuracy: 0.1427 - val_loss: 1.9459 - val_accuracy: 0.1476
Epoch 2/100
25/25 [==============================] - 2s 65ms/step - loss: 1.9456 - accuracy: 0.1484 - val_loss: 1.9463 - val_accuracy: 0.1426
Epoch 3/100
25/25 [==============================] - 2s 65ms/step - loss: 1.9452 - accuracy: 0.1503 - val_loss: 1.9464 - val_accuracy: 0.1427
Epoch 4/100
25/25 [==============================] - 2s 66ms/step - loss: 1.9445 - accuracy: 0.1564 - val_loss: 1.9457 - val_accuracy: 0.1444
Epoch 5/100
25/25 [==============================] - 2s 67ms/step - loss: 1.9433 - accuracy: 0.1652 - val_loss: 1.9448 - val_accuracy: 0.1490
Epoch 6/100
25/25 [==============================] - 2s 68ms/step - loss: 1.9413 - accuracy: 0.1747 - val_loss: 1.9429 - val_accuracy: 0.1601
Epoch 7/100
25/25 [==============================] - 2s 67ms/step - loss: 1.9379 - accuracy: 0.1872 - val_loss: 1.9409 - 

In [206]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate, Reshape, Conv1D, Flatten
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (21))

x = Reshape((-1, 1))(inputs)
print(x.shape)

x = Conv1D(filters = 1, kernel_size = 3, strides = 3)(x)
print(x.shape)
x = Flatten()(x)
x = Dense(1000, 'relu')(x)
x = Dense(1000, 'relu')(x)
outputs = Dense(7, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10000,100,validation_data=(Xt,Yt))

(None, 21, 1)
(None, 7, 1)
Epoch 1/100
25/25 [==============================] - 5s 190ms/step - loss: 1.9467 - accuracy: 0.1457 - val_loss: 1.9450 - val_accuracy: 0.1519
Epoch 2/100
25/25 [==============================] - 5s 195ms/step - loss: 1.9362 - accuracy: 0.1843 - val_loss: 1.9134 - val_accuracy: 0.2169
Epoch 3/100
25/25 [==============================] - 5s 204ms/step - loss: 1.8169 - accuracy: 0.3027 - val_loss: 1.6439 - val_accuracy: 0.4559
Epoch 4/100
25/25 [==============================] - 5s 214ms/step - loss: 1.3934 - accuracy: 0.5427 - val_loss: 1.1170 - val_accuracy: 0.6194
Epoch 5/100
25/25 [==============================] - 5s 219ms/step - loss: 0.9154 - accuracy: 0.7457 - val_loss: 0.7280 - val_accuracy: 0.7901
Epoch 6/100
25/25 [==============================] - 6s 227ms/step - loss: 0.6333 - accuracy: 0.8313 - val_loss: 0.5327 - val_accuracy: 0.8555
Epoch 7/100
25/25 [==============================] - 6s 232ms/step - loss: 0.4782 - accuracy: 0.8782 - val_loss: 0.

# Conclusion: 1 MLP layer after Conv1D gives about 75% test accuracy, and 2 MLP layers after Conv1D gives about 99% test accuracy. A tremendous improvement!

# What about the 22nd bit being hidden?

To evaluate 22nd bit, we need to use chunks of 6 instead, as chunks of 3 would leave the 22nd bit on its own and the neural network would not have any idea how to evaluate it.

In [216]:
import tensorflow as tf, numpy as np

# hyperparameters here
divisor = 7

# convert a number into binary
def int2bits(i,fill=24): 
    return list(map(int,bin(i)[2:].zfill(fill)))

def bits2int(b):
    return sum(i*2**n for n,i in enumerate(reversed(b)))

# Data. 
I = np.random.randint(0,2**21,size=(250000,))
X = np.array(list(map(int2bits, I))) 
Y = np.array([int2bits(2**i,divisor) for i in I % divisor])

# Test Data. 
It = np.random.randint(2**21,2**22,size=(10000,))
Xt = np.array(list(map(int2bits,It)))
Yt = np.array([int2bits(2**i,divisor) for i in It % divisor])

In [217]:
# Model.
from tensorflow.keras.layers import Dense, Input, Concatenate, Reshape, Conv1D, Flatten
from tensorflow.keras import Model


### Change the model architecture here
########################################################
inputs = Input(shape = (24))

x = Reshape((-1, 1))(inputs)
print(x.shape)

x = Conv1D(filters = 1, kernel_size = 6, strides = 6)(x)
print(x.shape)
x = Flatten()(x)
x = Dense(1000, 'relu')(x)
x = Dense(1000, 'relu')(x)
outputs = Dense(7, 'softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

########################################################

model.compile('adam','categorical_crossentropy',['accuracy'])

# Train (report the final score at the 20th epoch)
model.fit(X,Y,10000,100,validation_data=(Xt,Yt))

(None, 24, 1)
(None, 4, 1)
Epoch 1/100
25/25 [==============================] - 7s 270ms/step - loss: 1.9487 - accuracy: 0.1424 - val_loss: 1.9457 - val_accuracy: 0.1447
Epoch 2/100
25/25 [==============================] - 7s 275ms/step - loss: 1.9461 - accuracy: 0.1438 - val_loss: 1.9466 - val_accuracy: 0.1470
Epoch 3/100
25/25 [==============================] - 6s 257ms/step - loss: 1.9460 - accuracy: 0.1446 - val_loss: 1.9462 - val_accuracy: 0.1420
Epoch 4/100
25/25 [==============================] - 6s 253ms/step - loss: 1.9458 - accuracy: 0.1442 - val_loss: 1.9462 - val_accuracy: 0.1464
Epoch 5/100
25/25 [==============================] - 7s 266ms/step - loss: 1.9458 - accuracy: 0.1460 - val_loss: 1.9462 - val_accuracy: 0.1439
Epoch 6/100
25/25 [==============================] - 7s 281ms/step - loss: 1.9457 - accuracy: 0.1461 - val_loss: 1.9461 - val_accuracy: 0.1377
Epoch 7/100
25/25 [==============================] - 8s 301ms/step - loss: 1.9456 - accuracy: 0.1470 - val_loss: 1.

# Conclusion

We use chunks of 6 and get a test accuracy of 98%! What an achievement.

Overall, the modulo experiments have been interesting. It shows that the neural network cannot approximate any function easily, especially if it is out-of-sample.

In order to evaluate out-of-sample test cases, either one-shot learning or few-shot learning is required, with an expressive enough network structure.

For zero-shot learning, it seems that the structure to evaluate the unknown data needs to be inside the network. Here, for modulo, the Conv1D to evaluate 3 bits at a time is important. For CNNs, zero-shot learning should also be possible if the general concept of the image processing can be done on generic data, and not contingent on just a particular training set.

Moving on, I believe existing neural networks are not sufficient for learning concepts. It is more suited for pattern recognition and generalizing within a known domain. Perhaps we would need to do graphical models to better mimic the structure of the various functional domains within the brain, coupled with memory in order to link these concepts together.

I am keen to see how to improve the neural networks to do this. Current neural networks are a first step, but they are not the answer yet.